# Iteration 3: Sample length of stay on the ASU

## Plain English Summary

In the model length of stay is determined not just by patient type, but also patient destination post ASU.  For example Early Supported Discharge greatly reduces length of stay.  Now that we have sampled the destination we can sample the length of stay from Lognormal distributions.



## LLM Prompt

::::{admonition} LLM Prompt
:::{include} inputs/03_iteration.md
:::
::::

## Date and time of input to LLM

> 1500 2nd March 2024

## Observations on the LLM Output

* Note the input above had to be modified in the following ways:
  * "Do not use the yield statement when calling this method." - originally the code included the `yield` keyword before calling the acute_treatment process code. This meant arrival generator code was incorrect (wrong IAT).

* The `acute_treatment` functions appear to have been created correctly.

* Sampling from the lognormal distribution is not correct. I.e. it does not correctly convert from Lognormal mean and std to the underlying normal distribution.  It also mistakenly takes the natural logarithm. 

```python
length_of_stay = np.random.lognormal(mean=np.log(7.4), sigma=np.log(8.6))
```


## Sources reported by the LLM

* None given - Perplexity appears to have stopped providing them.

## LLM Output

In [3]:
import simpy
import numpy as np

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            length_of_stay = np.random.lognormal(mean=np.log(7.4), sigma=np.log(8.6))
        elif patient_type == "ESD":
            length_of_stay = np.random.lognormal(mean=np.log(4.6), sigma=np.log(4.8))
        else:
            length_of_stay = np.random.lognormal(mean=np.log(7.0), sigma=np.log(8.7))
        
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")

    def tia_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(1.8), sigma=np.log(5.0))
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")

    def neuro_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(4.0), sigma=np.log(5.0))
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")

    def other_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(3.8), sigma=np.log(5.2))
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            self.env.process(self.stroke_acute_treatment(post_asu_destination))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)


Stroke patient 1 arrived at 0.403483230784245 days and will go to ESD
Stroke patient 2 arrived at 1.509514039696369 days and will go to Other
Stroke patient 3 arrived at 1.8093710990995016 days and will go to Other
Stroke patient 4 arrived at 1.9857599507820036 days and will go to Other
Stroke patient 4 finished treatment at 2.3559169062966694 days
Stroke patient 5 arrived at 2.856399686259792 days and will go to Other
Stroke patient 6 arrived at 3.1164734278373953 days and will go to Other
Stroke patient 6 finished treatment at 3.262074023921837 days
Stroke patient 6 finished treatment at 5.187124477334998 days
Stroke patient 7 arrived at 5.635989197526297 days and will go to Other
Stroke patient 7 finished treatment at 6.343612476675041 days
Stroke patient 8 arrived at 6.874399024619795 days and will go to Other
Stroke patient 8 finished treatment at 7.303093242084632 days
Stroke patient 9 arrived at 7.679112190545811 days and will go to Rehab
Other patient 10 arrived at 8.1712797831

## Testing

* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods




### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [4]:
M = 10_000_000

In [5]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [6]:
test_1(M)

Stroke patient 1 arrived at 0.13567587700697534 days and will go to Other
Stroke patient 2 arrived at 1.9190524554811303 days and will go to ESD
Stroke patient 3 arrived at 4.241828991131646 days and will go to Rehab
Stroke patient 3 finished treatment at 4.584420825818405 days
Stroke patient 4 arrived at 5.049347442469196 days and will go to Other
Stroke patient 5 arrived at 5.185521683402278 days and will go to ESD
Stroke patient 6 arrived at 5.320905253717008 days and will go to Other
Stroke patient 6 finished treatment at 6.699957574875909 days
Stroke patient 7 arrived at 6.840254347443167 days and will go to Other
Stroke patient 8 arrived at 7.479125463163724 days and will go to Other
Stroke patient 9 arrived at 7.669193930899683 days and will go to ESD
Stroke patient 10 arrived at 7.697815106813695 days and will go to Other
Stroke patient 10 finished treatment at 8.003938723908648 days
Stroke patient 11 arrived at 10.035186497167155 days and will go to Other
Stroke patient 12 arr

### Extreme value test 2

* The code is again modified so that all patient typs have a mean inter-arrival rate of $M$

**Result: PASS**

In [7]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [8]:
test_2(M)

0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [9]:
TEN_DAYS = 10

In [10]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    


In [11]:
test_3(TEN_DAYS)

Complex Neuro patient 1 arrived at 0.4165700767329693 days
TIA patient 2 arrived at 1.458999768212003 days
Complex Neuro patient 3 arrived at 1.8416075710856825 days
Complex Neuro patient 4 arrived at 1.8598756172460236 days
Stroke patient 5 arrived at 2.058897715488348 days and will go to Other
Stroke patient 5 finished treatment at 3.114092180818022 days
Stroke patient 6 arrived at 3.2949280227168583 days and will go to Other
Stroke patient 6 finished treatment at 3.986162020063869 days
TIA patient 6 finished treatment at 5.709168532319603 days
Complex Neuro patient 6 finished treatment at 5.7802719021233395 days
Other patient 7 arrived at 7.275041589841601 days
TIA patient 8 arrived at 7.46684344977396 days
Other patient 8 finished treatment at 7.856854676741754 days
TIA patient 9 arrived at 7.917343725366996 days
Stroke patient 10 arrived at 9.13130945157254 days and will go to ESD
Stroke patient 11 arrived at 9.199924678129177 days and will go to Other
Stroke patient 11 finished t